In [86]:
import pandas as pd
import numpy as np
import os

In [173]:
# CA hourly electricity consumption in MWh, 2015-2022
ca_hourly_demand = pd.read_csv('data/ca_hourly_demand.csv')

# CA monthly CO2 emissions in MTCO2, 2014-2022
ca_monthly_emissions = pd.read_csv('data/ca_co2_emissions_monthly.csv')

# CA total EV registrations, 2010-2020
total_ca_num_evs = pd.read_csv('data/ca_ev_registrations_all.csv')

# CA yearly EV registrations, 2010-2020
ca_num_evs = pd.read_csv('data/ca_ev_yearly_registrations.csv')

/var/folders/_8/5w6s7qdn2tn4_tq1r88gqw040000gn/T/ipykernel_92568/2916958705.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  total_ca_num_evs = pd.read_csv('data/ca_ev_registrations_all.csv')


In [172]:
DATE_START_IDX = 10
DATE_END_IDX = 18
all_data = ca_hourly_demand
all_data["emissions"] = ""
all_data["registered EVs"] = ""

path_name = 'data/ca_co2_emissions_daily'
cwd = os.path.abspath(path_name)
file_list = os.listdir(cwd)

# Add hourly emissions values from 5-minute interval files
for file in file_list:
    curr_daily_file = pd.read_csv(path_name + "/" + file)
    curr_date = file[DATE_START_IDX:DATE_END_IDX]
    curr_year = int(curr_date[0:4])
    num_evs = 0
    if curr_year in list(ca_num_evs["Registration Valid Date"]):
        num_evs = ca_num_evs.loc[ca_num_evs['Registration Valid Date'] == curr_year, 'Unique Vehicles'].tolist()[0]
    for i in range(24):
        string_hour = str(i)
        if len(string_hour) < 2:
            string_hour = "0" + string_hour
        full_date = curr_date + "T" + string_hour + "Z"
        all_data.loc[all_data['date'] == full_date, 'emissions'] = curr_daily_file[string_hour + ":00"][0]
        all_data.loc[all_data['date'] == full_date, 'registered EVs'] = num_evs

all_data = all_data[all_data.emissions != ""]
all_data.to_csv('data/all_data.csv')

all_data.head()

,date,demand,emissions,registered EVs
0,20221127T23Z,23637,6655,test
1,20221127T22Z,23599,7323,test
2,20221127T21Z,23323,7393,test
3,20221127T20Z,23718,7321,test
4,20221127T19Z,25038,7519,test
